In [1]:
import os
import sys
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output

In [2]:
# Helper function
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)
        
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()

In [3]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

### Load numpy array

In [4]:
all_npy = np.load('./npy/all_4876.npy')
movie_genre = np.load('./npy/movie_genre.npy')
usr_following = np.load('./npy/user_followings.npy')
usr_genre = np.load('./npy/user_genre.npy')

print('All features:', all_npy.shape)
print('Movie genre:', movie_genre.shape)
print('User following:', usr_following.shape)
print('User genre:', usr_genre.shape)

All features: (165, 4876)
Movie genre: (165, 20)
User following: (1582, 165)
User genre: (1582, 20)


In [5]:
usr_nb = len(usr_following) # the number of users
movie_nb = len(movie_genre)  # the number of movies

print(usr_nb, movie_nb)

usr_test_amount = 150
movie_test_amount = 32

print(usr_test_amount, movie_test_amount)

1582 165
150 32


### Normalize usr_genre

In [6]:
usr_genre_norm = np.zeros(usr_genre.shape)
for i in range(len(usr_genre)):
    usr_genre_norm[i] = usr_genre[i]/np.max(usr_genre[i])
print(usr_genre_norm.shape)
# print('Before:', usr_genre)
# print('After:', usr_genre_norm)

(1582, 20)


# Training & testing split

## Prepare

In [7]:
#The number of followers for each movie
moive_followers = np.sum(usr_following, axis=0)
# print(moive_followers)

print('Min number of followers:', np.min(moive_followers))
print('Max number of followers:', np.max(moive_followers))
print('Avg of followers:', np.mean(moive_followers))

asc = np.sort(moive_followers)
# print(asc)
desc = np.flip(asc)
# print(desc)

over5 = 0
for num in moive_followers:
    if num >= 5:
        over5 += 1
print('The num of followers over 5:', over5)

Min number of followers: 1
Max number of followers: 520
Avg of followers: 142.0969696969697
The num of followers over 5: 163


In [8]:
#The number of following movie for each user
each_user = np.sum(usr_following, axis=1)
# print(each_user)

print('Min number of followings:', np.min(each_user))
print('Max number of followings:', np.max(each_user))
print('Avg of followers:', np.mean(each_user))

asc = np.sort(each_user)
# print(each_user)
# print(asc)
desc = np.flip(asc)
# print(desc)

Min number of followings: 10
Max number of followings: 133
Avg of followers: 14.820480404551201


In [9]:
print('Over 10:', np.sum(each_user >= 10))
print('Over 12:', np.sum(each_user >= 12))
print('Over 14:', np.sum(each_user >= 14))
print('Over 16:', np.sum(each_user >= 16))
print('Over 18:', np.sum(each_user >= 18))
print('Over 20:', np.sum(each_user >= 20))

Over 10: 1582
Over 12: 937
Over 14: 613
Over 16: 440
Over 18: 315
Over 20: 229


In [10]:
# over12_idx = np.nonzero(each_user >= 12)[0]
# over12_idx.shape

## Setup 

In [11]:
usr_idx = [i for i in range(len(usr_following))]
print(len(usr_idx))

random.seed(42)
test_idx = sorted(random.sample(usr_idx, usr_test_amount))
print(len(test_idx), test_idx[:10]) # 150 [13, 51, 54, 61, 65, 88, 93, 96, 114, 130]

1582
150 [13, 51, 54, 61, 65, 88, 93, 96, 114, 130]


In [12]:
train_t = []
train_f = []
test_t = []
test_f = []
test_pos = -1

for i in range(usr_nb):
    
    t_for_train = []
    f_for_train = []
    
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(movie_nb):
            if usr_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
                
        train_t.append(t_for_train)
        train_f.append(f_for_train)
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        
        for j in range(movie_nb):
            
            if usr_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        
        # random choose half true and half false for test 
        t_for_test = random.sample(temp_t, math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f, movie_test_amount-len(t_for_test))
        test_t.append(t_for_test)
        test_f.append(f_for_test)
        
        #the others for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t.append(t_for_train)
        train_f.append(f_for_train)

In [13]:
# train_t[i] 代表的是user i positive feedback
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_f))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 1582
The length of train_f: 1582
The length of test_t: 150
The length of test_f: 150


## Stat

In [14]:
#average num of following for training user
total_train = 0
for t in train_t:
    total_train += len(t)
avg = total_train / usr_nb
print('Training:', avg)

#average num of following for testing user
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test / usr_test_amount
print('Testing:', avg)

Training: 14.139064475347661
Testing: 7.1866666666666665


In [15]:
all_auxilary = [i for i in range(movie_nb)]

# Recommendation model

In [16]:
SAVE_NAME = 'MRM_ALL_Embedding200_L2_'
# LATENT_FOLDER = './latent_factor/MRM_ALL/Embedding200_L2_/'
# newPath(LATENT_FOLDER)

In [17]:
latent_dim = 128 # latent dims
ft_dim = all_npy.shape[1] # feature dims
embedding_dims = 200

print(latent_dim, ft_dim, embedding_dims)

128 4876 200


In [18]:
tf.reset_default_graph()

user = tf.placeholder(tf.int32,shape=(1,))
i = tf.placeholder(tf.int32, shape=(1,))
j = tf.placeholder(tf.int32, shape=(1,))

#多少個auxliary 
xf = tf.placeholder(tf.float32, shape=(None, ft_dim))
l_id = tf.placeholder(tf.int32, shape=(None,))
l_id_len = tf.placeholder(tf.int32,shape=(1,))
r = tf.placeholder(tf.float32,shape=(None,))
positive_id = tf.placeholder(tf.int32, shape=(None,))
positive_len = tf.placeholder(tf.int32,shape=(1,))

image_i = tf.placeholder(tf.float32, [1, ft_dim])
image_j = tf.placeholder(tf.float32, [1, ft_dim])

with tf.variable_scope("item_level"):
    user_latent = tf.get_variable("user_latent", [usr_nb, latent_dim],
                                  initializer=tf.random_normal_initializer(0,0.1,seed=3))
    item_latent = tf.get_variable("item_latent", [movie_nb, latent_dim],
                                  initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
    aux_item = tf.get_variable("aux_item", [movie_nb, latent_dim],
                               initializer=tf.random_normal_initializer(0,0.1,seed=3))
    
#     W1 = tf.get_variable("W1", [usr_nb, movie_nb, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
    Wu = tf.get_variable("Wu", [usr_nb, movie_nb, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
    Wy = tf.get_variable("Wy", [usr_nb, movie_nb, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
    Wa = tf.get_variable("Wa", [usr_nb, movie_nb, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
    Wv = tf.get_variable("Wv", [usr_nb, movie_nb, embedding_dims], initializer=tf.contrib.layers.xavier_initializer())
    Wve =  tf.get_variable("Wve", [embedding_dims, ft_dim], initializer=tf.contrib.layers.xavier_initializer())
    
    aux_new = tf.get_variable("aux_new", [1, latent_dim], initializer=tf.constant_initializer(0.0))

with tf.variable_scope('feature_level'):
    embedding = tf.get_variable("embedding", [embedding_dims,ft_dim],
                                initializer=tf.contrib.layers.xavier_initializer())
    Beta = tf.get_variable("beta", [usr_nb, embedding_dims],
                           initializer=tf.random_normal_initializer(0.01, 0.001, seed=10))
    
#lookup the latent factors by user and id
u = tf.nn.embedding_lookup(user_latent, user)
vi = tf.nn.embedding_lookup(item_latent, i)
vj = tf.nn.embedding_lookup(item_latent, j)

# w1 = tf.nn.embedding_lookup(W1, user)
wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user))
wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user))
wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user))
wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user))

beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [20]:
a_list = tf.Variable([])
q = tf.constant(0)

def att_cond(q,a_list):
    return tf.less(q,l_id_len[0])

def att_body(q,a_list):
    xfi = tf.expand_dims(xf[q],0) #(1,l)
    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
    
    a_list = tf.concat([a_list,[(tf.nn.relu(tf.matmul(wuui, u, transpose_b=True) +
                                            tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                            tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                            tf.matmul(wvui,tf.matmul(Wve,xfi, transpose_b=True)))[0][0])*r[q]]],0)
    q += 1
    return q, a_list

_, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

a_list_smooth = tf.add(a_list,0.0000000001)
a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

norm_par = [wu,wy,wa,wv]


wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
wv_be_relu = tf.matmul(wvui, tf.matmul(Wve,tf.expand_dims(xf[-1],0), transpose_b=True))

last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]

aux_np = tf.expand_dims(tf.zeros(latent_dim),0)
q = tf.constant(0)

def sum_att_cond(q,aux_np):
    return tf.less(q,l_id_len[0])

def sum_att_body(q,aux_np):
    aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
    q += 1
    return q, aux_np

_, aux_np = tf.while_loop(sum_att_cond, sum_att_body, [q,aux_np])

aux_part = tf.matmul(aux_np, vi, transpose_b=True)
#tf.print('aux attention:',aux_np)
aux_np += u #user_latent factor + sum (alpha*auxilary)
aux_new = tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np

latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

#矩陣中對應函數各自相乘
# ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

xuij = tf.subtract(xui,xuj)

l2_norm = tf.add_n([
            0.0001 * tf.reduce_sum(tf.multiply(u, u)),
            0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
            0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),
  
            0.01 * tf.reduce_sum(tf.multiply(wu, wu)),
            0.01 * tf.reduce_sum(tf.multiply(wy, wy)),
            0.01 * tf.reduce_sum(tf.multiply(wa, wa)),
            0.01 * tf.reduce_sum(tf.multiply(wv,wv)),
            
            0.001 * tf.reduce_sum(tf.multiply(beta,beta)),
            0.01 * tf.reduce_sum(tf.multiply(embedding,embedding))
            
          ])

loss = l2_norm - tf.log(tf.sigmoid(xuij)) # objective funtion
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
auc = tf.reduce_mean(tf.to_float(xuij > 0))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


## Training

In [21]:
print('Start time:', time.ctime())

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
loss_acc_list = []
t0 = time.time()

train_yes_id=[]

for q in range(10):
    print('Iteration:',q)
    train_auc = 0
    total_loss = 0
    xuij_auc = 0
    length = 0
    
    for z in range(usr_nb):
        writeProgress('Progress:', z, usr_nb)
        """
        yes 用來存放選擇到的YouTuber feature (for auxilary)
        yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
        r_3 用來存放user 對該YouTuber種類的偏好(取max)
        """
        yes = []
        yesr = []
        
        #選全部的Positive
        sample = random.sample(train_t[z],len(train_t[z]))
#         #選全部的電影
#         sample = all_auxilary
        
        #change
        r_3 = np.zeros(len(sample))
         
        for b in range(len(sample)):
            yes.append(all_npy[sample[b]])
            yesr.append(movie_genre[sample[b]] * usr_genre_norm[z])
        
        for b in range(len(yesr)):
            r_3[b]=max(yesr[b])
        #print('r_3:',r_3)
        
        yes = np.array(yes)
        
        # positive 
        train_t_sample = random.sample(train_t[z],len(train_t[z]))
        #print('number of positive feedback', len(train_t[z]))
        # negative
#         train_f_sample = random.sample(train_f[z],20)
        
        for ta in train_t_sample:
            #print(ta,'--> positive feedback')
            
            pos = sample.index(ta)
            
            image_1=np.expand_dims(all_npy[ta],0)
            train_f_sample = random.sample(train_f[z],10)
            
            for b in train_f_sample:
                image_2 = np.expand_dims(all_npy[b],0)
                
                _last_be_relu, _norm_par, _a_list, r3, _auc, _loss, _ = sess.run(
                    [last_be_relu, norm_par, a_list_smooth, a_list_soft, auc, loss, train_op], 
                    feed_dict={user: [z], i: [ta], j: [b], xf: yes , 
                               l_id:sample, l_id_len:[len(sample)],
                               positive_id: train_t[z], positive_len:[len(train_t[z])],
                               r:r_3, image_i:image_1, image_j:image_2})
                
                train_auc += _auc
                total_loss += _loss
                length += 1
        
#         np.save(LATENT_FOLDER + str(q) + '_' + str(z),_embedding)
    
    #print('mine:',xuij_auc/136)   
    #print('a_list_soft:',r3)
    print("{:<20}{}".format('total_loss', total_loss/length))
    print("{:<20}{}".format('train_auc:', train_auc/length))
    
    loss_acc_list.append([total_loss/length, train_auc/length, time.time()-t0])
    
    print('\tCurrent time:', time.ctime(), ' sec')
    print('==================================================')
    
print('Total cost time:',time.time()-t0, ' sec')

print('End time:', time.ctime())

Start time: Sun Mar  8 23:16:22 2020
Iteration: 0
total_loss          [[0.5738897]]
train_auc:          0.8062097639484979
	Current time: Mon Mar  9 01:57:32 2020  sec
Iteration: 1
total_loss          [[0.50706275]]
train_auc:          0.8459451001430616
	Current time: Mon Mar  9 04:33:44 2020  sec
Iteration: 2
total_loss          [[0.47694844]]
train_auc:          0.8656473533619456
	Current time: Mon Mar  9 07:08:57 2020  sec
Iteration: 3
total_loss          [[0.45689947]]
train_auc:          0.8761355507868384
	Current time: Mon Mar  9 09:44:21 2020  sec
Iteration: 4
total_loss          [[0.44103655]]
train_auc:          0.8845985336194564
	Current time: Mon Mar  9 12:21:18 2020  sec
Iteration: 5
total_loss          [[0.4338476]]
train_auc:          0.8889887339055794
	Current time: Mon Mar  9 14:58:23 2020  sec
Iteration: 6
total_loss          [[0.42840271]]
train_auc:          0.8921092632331903
	Current time: Mon Mar  9 17:31:08 2020  sec
Iteration: 7
total_loss          [[0.4233

In [22]:
for i in range(len(loss_acc_list)):
    print('Iteration:',i)
    print('loss=',loss_acc_list[i][0])
    print('acc=',loss_acc_list[i][1])
#     print('time=',loss_acc_list[i][2])
    print('==================================================')

Iteration: 0
loss= [[0.5738897]]
acc= 0.8062097639484979
Iteration: 1
loss= [[0.50706275]]
acc= 0.8459451001430616
Iteration: 2
loss= [[0.47694844]]
acc= 0.8656473533619456
Iteration: 3
loss= [[0.45689947]]
acc= 0.8761355507868384
Iteration: 4
loss= [[0.44103655]]
acc= 0.8845985336194564
Iteration: 5
loss= [[0.4338476]]
acc= 0.8889887339055794
Iteration: 6
loss= [[0.42840271]]
acc= 0.8921092632331903
Iteration: 7
loss= [[0.42332689]]
acc= 0.8960345135908441
Iteration: 8
loss= [[0.42093095]]
acc= 0.8981402002861231
Iteration: 9
loss= [[0.41788154]]
acc= 0.9000223533619456


# Get latent factor and Each weight

In [29]:
U, Y, A, E, Au, Ay, Aa, Av, Ave, B =sess.run([user_latent, item_latent, aux_item, embedding, Wu, Wy, Wa, Wv, Wve, Beta])

In [35]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:', Aa.shape)
print('Wv weight shape:', Av.shape)
print('Wve weight shape:', Ave.shape)
print('Embedding shape:', E.shape)
print('Beta shape:',B.shape)

User latent shape:  (1582, 128)
photo latent shape:  (165, 128)
Auxilary latent shape:  (165, 128)
Wu weight shape: (1582, 165, 128)
Wy weight shape: (1582, 165, 128)
Wa weight shape: (1582, 165, 128)
Wv weight shape: (1582, 165, 200)
Wve weight shape: (200, 4876)
Embedding shape: (200, 4876)
Beta shape: (1582, 200)


In [31]:
np.savez('./weight/' + SAVE_NAME + '.npz', 
         U=U, Y=Y, A=A, E=E, Wu=Au, Wy=Ay, Wa=Aa, Wv=Av,B=B,Wve=Ave)

# Testing Part

In [44]:
'''
usr_test_amount = 150
movie_test_amount = 32
'''

#with Embedding
result = np.zeros((usr_test_amount, movie_nb))
RS = np.zeros((usr_test_amount, movie_nb))
#test_idx --> Test 的 index

test_yes_id = []

for s in range(usr_test_amount):
    print(s, test_idx[s])

    yes = []
    sample = random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha = np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r = np.max(movie_genre[sample[a]] * usr_genre_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))

        alpha[a] = np.sum((relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T) +
                                np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T) +
                                np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T) +
                                np.dot(Ave[test_idx[s]],np.expand_dims(all_npy[sample[a]],0).T))))*r
    
        
    mul = np.zeros((1,latent_dim))
    
    print("{:<15}{}".format('alpha:', alpha))
    print("{:<15}{}".format('softmax alpha:', softmax(alpha)))
    print('==================================================')
    
    for i in range(len(sample)):
        mul += softmax(alpha)[i] * A[sample[i]] #attention alpha*Ai part 
    new_mul = mul + U[test_idx[s]]  #(U+auxilary)
    
    for k in range(movie_nb):
        result[s][k] = np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T) + np.dot(B[test_idx[s]], np.dot(E, all_npy[k].T))
        
#print(RS[s])

0 13
alpha:         [74.70322215  0.          0.          0.          0.          0.
 33.08789019]
softmax alpha: [1.00000000e+00 3.60414984e-33 3.60414984e-33 3.60414984e-33
 3.60414984e-33 3.60414984e-33 8.44677580e-19]
1 51
alpha:         [ 84.10766201 251.64912589 124.96599081   0.          59.46594414
   0.        ]
softmax alpha: [1.72848950e-073 1.00000000e+000 9.59872355e-056 5.13066016e-110
 3.43482528e-084 5.13066016e-110]
2 54
alpha:         [0. 0. 0. 0. 0. 0.]
softmax alpha: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
3 61
alpha:         [0. 0. 0. 0. 0.]
softmax alpha: [0.2 0.2 0.2 0.2 0.2]
4 65
alpha:         [  0.         145.24246666  77.89186303 174.69974724   0.
  98.80790755]
softmax alpha: [1.34543829e-76 1.61014724e-13 9.05461688e-43 1.00000000e+00
 1.34543829e-76 1.09797588e-33]
5 88
alpha:         [ 55.74230786  94.98787521  76.75199755 237.61000241 124.59009248
  21.27745926   0.          76.54024921 137.86060404]
softmax alpha: [1.0372031

IndexError: index 206 is out of bounds for axis 0 with size 200

In [45]:
#取出test的資料
testRS = np.zeros((usr_test_amount, movie_test_amount)) #shape 150*20
target = np.zeros((usr_test_amount, movie_test_amount))
#test_t 是true的
#test_f 是false的
        
for z in range(usr_test_amount):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1
        
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

In [46]:
print(target.shape, testRS.shape)

(150, 32) (150, 32)


In [47]:
sumtarget = 0
for i in range(len(target)):
    #print(np.sum(target[i]))
    sumtarget += np.sum(target[i])
print('num of positive data in testing:',sumtarget)
print('total testing data:', usr_test_amount * movie_test_amount)

num of positive data in testing: 1078.0
total testing data: 4800


In [48]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [49]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    #print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/100

print('avg_accuarcy for count_0:', avg_acc)
print(acc_0, total)

# Top n

In [52]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

## Top 1

In [53]:
correct = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),1) #取一個
    count_0_all.append(top_0)
    #print(np.sum(target[i]))
    #print(top_0)
    if top_0[0] < int(np.sum(target[i])):
        correct += 1

In [54]:
top1_prec = correct/len(testRS)
top1_recall = correct/(sumtarget)
print('prec ',top1_prec,'recall ',top1_recall)
print('F1_score:',F1_score(top1_prec,top1_recall))

prec  0.9333333333333333 recall  0.12987012987012986
F1_score: 0.22801302931596087


## Top 3

In [55]:
correct = 0
for i in range(len(testRS)):
    top_3 = topN(list(testRS[i]),3) #取一個
    count_0_all.append(top_3)
    #print(top_3)
    for j in range(len(top_3)):
        if top_3[j] < int(np.sum(target[i])):
            correct += 1

In [56]:
top3_prec = correct/(len(testRS)*3)
top3_recall = correct/(sumtarget)
print('prec ',top3_prec,'recall ',top3_recall)
print('F1_score:',F1_score(top3_prec,top3_recall))

prec  0.9155555555555556 recall  0.3821892393320965
F1_score: 0.5392670157068064


## Top 5

In [57]:
correct = 0
for i in range(len(testRS)):
    top_5 = topN(list(testRS[i]),5) #取一個
    count_0_all.append(top_5)
    #print(top_5)
    for j in range(len(top_5)):
        if top_5[j] < int(np.sum(target[i])):
            correct += 1

In [58]:
top5_prec = correct/(len(testRS)*5)
top5_recall = correct/(sumtarget)
print('prec ',top5_prec,'recall ',top5_recall)
print('F1_score:',F1_score(top5_prec,top5_recall))

prec  0.9173333333333333 recall  0.6382189239332097
F1_score: 0.7527352297592997


# pre_list

In [59]:
'''
usr_test_amount = 150
movie_test_amount = 16
'''
all_sort = []
pre_matrix = np.zeros(shape=(usr_test_amount, movie_test_amount))
for i in range(usr_test_amount):
    top_5 = topN(list(testRS[i]),5) #取一個
    #print(top_5)
    all_sort.append(topN(list(testRS[i]),len(testRS[i])))
    for j in range(len(top_5)):
        pre_matrix[i][top_5[j]] = 1

In [60]:
print(pre_matrix.shape, target.shape)

(150, 32) (150, 32)


# NDCG
* https://daiwk.github.io/posts/nlp-ndcg.html

In [61]:
#Ideal DCG，理想状况下的DCG。也就是说，相关性完全由高到低排序时算出的DCG：

def IDCG(ideal_list): #ideal_list example = [1,1,1,1,1,0,0,....]
    idcg = 0
    for i in range(len(ideal_list)):
        #print((2**true_list[i]-1),math.log2(i+2))
        idcg += (2**ideal_list[i]-1)/math.log2(i+2)
    #print('idcg',idcg)
    return idcg

def DCG(prec_list): #找出前n名的[1,1,1,0,...]
    dcg = 0
    for i in range(len(prec_list)):
        dcg += (2**prec_list[i]-1)/math.log2(i+2)
    #print('dcg',dcg)
    return dcg

In [62]:
total_ndcg = 0
num_ndcg = 5
for m in range(usr_test_amount):
    idcg = IDCG([1]*num_ndcg)
    pre_list = []
    for s in all_sort[m][:num_ndcg]:
        #print(s)
        #print(target[m][s])
        pre_list.append(target[m][s])
    dcg = DCG(pre_list)
    ndcg = dcg/idcg
    #print(ndcg)
    total_ndcg += ndcg
avg_ndcg = total_ndcg/usr_test_amount
print('NDCG:',avg_ndcg)

NDCG: 0.9199021358837534


# MAP

In [63]:
from sklearn.metrics import average_precision_score

In [64]:
total_prec = 0
for u in range(usr_test_amount):
    y_true = target[u]
    y_scores = pre_matrix[u]
    total_prec += average_precision_score(y_true, y_scores)
    
MAP = total_prec/usr_test_amount

print('MAP:', MAP)

MAP: 0.7533307686866504
